In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import dask.dataframe as dd

import warnings
warnings.filterwarnings('ignore')

Load the data...

In [53]:
df = pd.read_csv('../csv/credit_card_fraud_2019_q1.csv')
#df = pd.read_csv('csv/credit_card_fraud_sample_10K.csv')

In [54]:
# Set Pandas options to display full rows
pd.set_option('display.max_colwidth', None)  # Show full column contents
pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.width', 1000)  # Adjust width for full display
#pd.set_option('display.max_rows', None)  # Show all rows (use with caution for large datasets)

In [55]:
df.head()

,Unnamed: 0,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long,quarter
0,44,053-27-1953,4499269417632698520,Elizabeth,Donovan,F,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,42c903534492148a68aead5538c61c89,2019-01-18,05:32:48,1547769768,gas_transport,75.77,0,fraud_Ledner-Pfannerstill,32.862965,-95.719060,1
1,45,053-27-1953,4499269417632698520,Elizabeth,Donovan,F,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,a92dad553afa8372e1ed123a6d287245,2019-02-16,02:11:07,1550263267,gas_transport,69.04,0,fraud_Torp-Labadie,33.160655,-97.025090,1
2,46,053-27-1953,4499269417632698520,Elizabeth,Donovan,F,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,63ef657aa1d0289c618b731150abbed7,2019-03-24,11:50:13,1553408413,gas_transport,52.73,0,fraud_Huels-Hahn,32.454127,-95.627965,1
3,47,053-27-1953,4499269417632698520,Elizabeth,Donovan,F,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,49d82b0d4a06f9533dd82bd759f4ab3b,2019-01-30,06:52:29,1548811349,gas_transport,85.55,0,"fraud_Zieme, Bode and Dooley",33.479622,-95.990827,1
4,51,053-27-1953,4499269417632698520,Elizabeth,Donovan,F,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,3f292e4616045c4cb570dbcc15c4be4e,2019-02-24,11:38:44,1550988524,gas_transport,37.01,0,fraud_Conroy-Cruickshank,32.871351,-96.961381,1


In [56]:
df.isna().sum()

Unnamed: 0    0
ssn           0
cc_num        0
first         0
last          0
gender        0
street        0
city          0
state         0
zip           0
lat           0
long          0
city_pop      0
job           0
dob           0
acct_num      0
profile       0
trans_num     0
trans_date    0
trans_time    0
unix_time     0
category      0
amt           0
is_fraud      0
merchant      0
merch_lat     0
merch_long    0
quarter       0
dtype: int64

Clean the Data
1. consider drop first/last name, ssn, cc_num, trans_num etc...
2. explore locational data e.g zip lat/long, city etc... TBD replace with regions??
3. reorganize date & time data:
    3.1 work day vs weekend
    3.2 TBD: split hour segment, work/sleep/free hours
4. set 4 age groups youngest to oldest (18-199), teenager (-18) young_adult (19-25), adult (26-65), retired (...)
    4.1 TBD treat different for men & women?? retirement age 66(M), 63(F)

5. categorize: category,          

6. explore: amt, dob? 


In [57]:
df.nunique()


Unnamed: 0    3226281
ssn             17722
cc_num          17722
first             686
last             1000
gender              2
street          17722
city             4929
state              51
zip              9611
lat              9301
long             9480
city_pop         5903
job               639
dob             12664
acct_num        17722
profile            12
trans_num     3226281
trans_date         90
trans_time      51155
unix_time     2131994
category           14
amt             71968
is_fraud            2
merchant          693
merch_lat     2934870
merch_long    3094870
quarter             1
dtype: int64

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3226281 entries, 0 to 3226280
Data columns (total 28 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ssn         object 
 2   cc_num      int64  
 3   first       object 
 4   last        object 
 5   gender      object 
 6   street      object 
 7   city        object 
 8   state       object 
 9   zip         int64  
 10  lat         float64
 11  long        float64
 12  city_pop    int64  
 13  job         object 
 14  dob         object 
 15  acct_num    int64  
 16  profile     object 
 17  trans_num   object 
 18  trans_date  object 
 19  trans_time  object 
 20  unix_time   int64  
 21  category    object 
 22  amt         float64
 23  is_fraud    int64  
 24  merchant    object 
 25  merch_lat   float64
 26  merch_long  float64
 27  quarter     int64  
dtypes: float64(5), int64(8), object(15)
memory usage: 689.2+ MB


In [59]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [60]:
print(df['gender'].value_counts())
df['gender'] = df['gender'].map({'M': 1, 'F': 0})
print(df['gender'].value_counts())


gender
F    1671372
M    1554909
Name: count, dtype: int64
gender
0    1671372
1    1554909
Name: count, dtype: int64


In [61]:

df['trans_date'] = pd.to_datetime(df['trans_date'])
df['trans_time'] = pd.to_datetime(df['trans_time'])

df['trans_year'] = df['trans_date'].dt.year
df['trans_month'] = df['trans_date'].dt.month
df['trans_day'] = df['trans_date'].dt.day

df['trans_hour'] = df['trans_time'].dt.hour
df['trans_minute'] =df['trans_time'].dt.minute
#df['trans_second'] =df['trans_time'].dt.second


df['dob'] = pd.to_datetime(df['dob'])
df['age'] = df['trans_year']  - df['dob'].dt.year


In [62]:
#age groups

def age_group(age):
    if age < 15:
        return 'child'
    elif age < 20:
        return 'teenager'
    elif age < 40:
        return 'young_adult'
    elif age < 60:
        return 'middle_aged'
    elif age < 80:
        return 'senior'
    else:
        return 'elderly'

df['age_group'] = df['age'].apply(age_group)

In [63]:
df['age_group'].value_counts()

age_group
young_adult    1393463
middle_aged     996144
senior          399290
teenager        279675
elderly         118793
child            38916
Name: count, dtype: int64

In [64]:
df['ssn'] = df['ssn'].astype('string')
df['street'] = df['street'].astype('string')
df['city'] = df['city'].astype('string')
df['state'] = df['state'].astype('string')
df['profile'] = df['profile'].astype('string')
df['trans_num'] = df['trans_num'].astype('string')
df['category'] = df['category'].astype('string')
df['merchant'] = df['merchant'].astype('string')



In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3226281 entries, 0 to 3226280
Data columns (total 35 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   ssn           string        
 2   cc_num        int64         
 3   first         object        
 4   last          object        
 5   gender        int64         
 6   street        string        
 7   city          string        
 8   state         string        
 9   zip           int64         
 10  lat           float64       
 11  long          float64       
 12  city_pop      int64         
 13  job           object        
 14  dob           datetime64[ns]
 15  acct_num      int64         
 16  profile       string        
 17  trans_num     string        
 18  trans_date    datetime64[ns]
 19  trans_time    datetime64[ns]
 20  unix_time     int64         
 21  category      string        
 22  amt           float64       
 23  is_fraud      int64         
 24

In [66]:
# set area
#df['profile'] = df['profile'].str.replace('.json', '') 
# Extract the area (last word before .json)
df['area'] = df['profile'].str.extract(r'_(rural|urban)\.json')

print(df['area'].value_counts())


area
urban    3058679
rural     167602
Name: count, dtype: Int64


In [ ]:
#potential dropouts
#['ssn','cc_num', 'trans_num', 'first','last','dob']


In [ ]:
df.head(3)

,id,ssn,cc_num,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,acct_num,profile,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long,quarter,trans_year,trans_month,trans_day,trans_hour,trans_minute,age,age_group,area
0,44,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,42c903534492148a68aead5538c61c89,2019-01-18,2025-03-09 05:32:48,1547769768,gas_transport,75.77,0,fraud_Ledner-Pfannerstill,32.862965,-95.719060,1,2019,1,18,5,32,20,young_adult,urban
1,45,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,a92dad553afa8372e1ed123a6d287245,2019-02-16,2025-03-09 02:11:07,1550263267,gas_transport,69.04,0,fraud_Torp-Labadie,33.160655,-97.025090,1,2019,2,16,2,11,20,young_adult,urban
2,46,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,63ef657aa1d0289c618b731150abbed7,2019-03-24,2025-03-09 11:50:13,1553408413,gas_transport,52.73,0,fraud_Huels-Hahn,32.454127,-95.627965,1,2019,3,24,11,50,20,young_adult,urban
3,47,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,49d82b0d4a06f9533dd82bd759f4ab3b,2019-01-30,2025-03-09 06:52:29,1548811349,gas_transport,85.55,0,"fraud_Zieme, Bode and Dooley",33.479622,-95.990827,1,2019,1,30,6,52,20,young_adult,urban
4,51,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,3f292e4616045c4cb570dbcc15c4be4e,2019-02-24,2025-03-09 11:38:44,1550988524,gas_transport,37.01,0,fraud_Conroy-Cruickshank,32.871351,-96.961381,1,2019,2,24,11,38,20,young_adult,urban
5,52,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,96a2f43c9cd18402a6837161cb44107f,2019-02-13,2025-03-09 01:57:41,1550003261,gas_transport,51.85,0,fraud_Prohaska-Murray,33.854362,-96.435575,1,2019,2,13,1,57,20,young_adult,urban
6,54,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,e0bea4614ed29c8a7592706515307e07,2019-02-23,2025-03-09 01:50:21,1550866821,gas_transport,48.49,0,"fraud_Jenkins, Hauck and Friesen",32.455503,-96.539877,1,2019,2,23,1,50,20,young_adult,urban
7,55,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,135e1dddfc7ebebcbdd49659cb86d3c8,2019-01-14,2025-03-09 02:04:03,1547411643,gas_transport,52.53,0,"fraud_Eichmann, Bogan and Rodriguez",33.144116,-95.915190,1,2019,1,14,2,4,20,young_adult,urban
8,56,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,646543104414,young_adults_female_urban.json,c1aa294fcf769b20ab509578a00a0bbb,2019-03-21,2025-03-09 07:00:20,1553131820,gas_transport,81.23,0,"fraud_Raynor, Feest and Miller",33.044018,-95.741208,1,2019,3,21,7,0,20,young_adult,urban
9,57,053-27-1953,4499269417632698520,Elizabeth,Donovan,0,1388 Thompson Station,Rowlett,TX,75089,32.9346,-96.5544,54963,English as a second language teacher,1999-06-20,64654310